In [141]:
import pandas as pd
import yaml
import json
import os
import subprocess


In [142]:
DATA_FILE='export_basis_AMPs_aangepast_20220425.xlsx'
OUT_FOLDER_VMP="output-vmp"
OUT_FOLDER_MP="output-MP"
OUT_FOLDER_PMP="output-PMP"
OUT_FOLDER_VMP_PROFILE="output-vmp-profile"
DATA_FILE2='export_basis_AMPPs_aangepast_20220425.xlsx'
REPORT_FILE="example-report-ampp.csv"

In [143]:
data=pd.read_excel(io=DATA_FILE,sheet_name='Blad1')
data2=pd.read_excel(io=DATA_FILE2,sheet_name='Blad1')

In [144]:
#Create folder if they do not exist
if not os.path.exists(OUT_FOLDER_VMP):
    os.makedirs(OUT_FOLDER_VMP)
if not os.path.exists(OUT_FOLDER_MP):
    os.makedirs(OUT_FOLDER_MP)
if not os.path.exists(OUT_FOLDER_PMP):
    os.makedirs(OUT_FOLDER_PMP)
if not os.path.exists(OUT_FOLDER_VMP_PROFILE):
    os.makedirs(OUT_FOLDER_VMP_PROFILE)   

In [145]:
def create_yaml(x):
    """
    create yaml file from dataframe (CSV)
    """
    doc={"type":"Tree","rows":[{"label":"CSV","rows":[]}]}

    for col in x.columns:
        l_id=col
        doc["rows"][0]["rows"].append({"label":col,"id":l_id})

    doc["rowHeight"]=25
    doc["headerHeight"]=50
    doc["headerText"]=DATA_FILE.split('.')[0]
    doc["id"]=DATA_FILE.split("/")[-1].split(".")[0].replace(" ","_")
    color="#34ebcf" 
    doc["attrs"]={"header":{"fill":color}}
    doc2=[doc]
    file=DATA_FILE.split(".")
    ff2 = open(file[0]+".yaml", 'w+')
    ffjson = open(file[0]+ ".json", "w+")
    json.dump(doc2, ffjson)
    yaml.dump(doc2,ff2,default_flow_style=False, sort_keys=False,indent=2)
    return "ok"

In [146]:
create_yaml(x=data)

'ok'

In [147]:
def create_substance(x):
    sbs={"resourceType": "MedicationKnowledge"}

   # sbs["identifier"]=[]
   # sbs["identifier"].append({"system":"http://www.belgium.be/susbtance","value":int(x["ntlidentifier"])})
    sbs["status"]="active"
    sbs["domain"]="human"
    sbs["name"]={}
    sbs["name"]["productName"]=x["basis"]
    sbs["name"]["type"]="generic"
    sbs["granularity"]="susbtance"
   
    #sbs["drugType"]= molecule_type if "molecule_type" in locals() else None
    if not pd.isna(x["atc"]):
        sbs["classification"]=[]
        sbs["classification"].append({"system":"http://who.org/ATC","value":x["atc"]})
   # if "indication" in locals() and indication is not None:
   #     sbs["classification"].append({"system":"http://who.org/indication","value":indication})

    #time.sleep(0.1)
    print(sbs)
    return sbs


In [148]:
def validate_artifact(art,ig=None):
    """
    for all ok:
        grep this 'Success: 0 errors, 0 warnings, 1 notes
                        Information: All OK'
    for errors: 
        the output goes to expcetion and returns similar to:
        *FAILURE*: 1 errors, 1 warnings, 1 notes\n  
        Error @ MedicationKnowledge.intendedRoute[0].coding[0].code (line 1, col550): 
        Error parsing JSON: the primitive value must be a string\n  Information @ MedicationKnowledge.meta.profile[0] (line 1, col137): 
        Canonical URL 'http://hl7.org/fhir/us/example/StructureDefinition/PharmaceuticalProduct' does not resolve\n  Warning @ MedicationKnowledge.meta.profile[0] (line 1, col2): Profile reference 'http://hl7.org/fhir/us/example/StructureDefinition/PharmaceuticalProduct' has not been checked because it is unknown, and fetching it resulted in the error org.hl7.fhir.r4.utils.client.EFhirClientException: Error parsing response message: This does not appear to be a FHIR resource (wrong namespace 'http://www.w3.org/1999/xhtml') (@ /)\n"
    """
    result={}
    command= ["java", "-jar", "/Users/joaoalmeida/Desktop/transformaçao-meds-be/validator_cli.jar","/Users/joaoalmeida/Desktop/transformaçao-meds-be/"+art]
    if ig:
        command=["java", "-jar", "/Users/joaoalmeida/Desktop/transformaçao-meds-be/validator_cli.jar","/Users/joaoalmeida/Desktop/transformaçao-meds-be/"+art,"-ig",ig]
    try:
        output = subprocess.check_output(command, stderr=subprocess.PIPE)
        nopt=output.decode('UTF-8')
     #   print(nopt)
        result["status"]="OK"

        result["message"]=nopt.split("Success: ")[-1]
    except subprocess.CalledProcessError as e:
       # print("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
        nopt=e.output.decode("UTF-8").split("FAILURE*: ")[-1]
       # print(e.output.decode("UTF-8"))
        result["status"]="NOK"
        result["message"]=nopt
       # print(e.output)
    return result

In [149]:
def save_files(folder,file_name,validation,artifact):
  
  if folder:
#  ffjson = open(file_name, "w+")
    with open(folder+"/"+file_name, 'w+') as f:
      f.write(json.dumps(artifact,indent=4))
  # print( os.path.exists(file_name))
  if validation:
    outcome=validate_artifact(folder+"/"+file_name)
    return outcome
  return None
   

In [150]:
def create_pharmprod(data,folder=None,validation=False):
    #print(data)
    artifact={}
    artifact["resourceType"]="MedicationKnowledge"


    artifact["status"]="active"
    artifact["synonym"]=[data["vmp_name"] ]
    
    artifact["ingredient"]=[]
    denominator={"value":1} #must have 1 for ratio...
    ingredient={}

    ingredient["itemCodeableConcept"]={"coding":[{"display":data["basis_substance"]}]}
    ingredient["isActive"]=True
   # print(data["strenght_nominator_value_low_limit"],data["strength_nominator_unit"])
    numerator={"value":data["strenght_nominator_value_low_limit"],"unit":data["strength_nominator_unit"]}
    strength={"numerator":numerator}
      
    if data["strenght_denominator_value_low_limit"]=="":
     # print(data["strenght_denominator_value_low_limit"])
      denominator={"value":int(data["strenght_denominator_value_low_limit"]),"unit":data["strength_denominator_unit"]}
    strength["denominator"]=denominator
    ingredient["strength"]=strength
    artifact["ingredient"].append(ingredient)

    artifact["doseForm"] ={ ##create link with snomed or idmp
    "coding": [
      {
        "code": str(data["edqmid"]),
        "system": "http://standardterms.edqm.eu",
        "display": data["edqmform"]
      }
    ]
  }
    route=[]
   
    route.append( {
      "coding": [
        {
          "code": str(data["edqm_roa_id"]),
          "system": "http://standardterms.edqm.eu",
          "display": data["roa_en"]
        }
      ]
    })
    artifact["intendedRoute"]=route
    file_name=str(data["vmp_name"]).replace("/",".").replace(".","").replace("(","").replace(")","").replace(" ","_")+ ".json"

    outcome=save_files(folder,file_name,validation,artifact)
    if outcome:
        return outcome["status"],outcome["message"]

    return artifact

In [151]:
data.iloc[0:2].apply(create_pharmprod,axis=1,folder=OUT_FOLDER_VMP)

0    {'resourceType': 'MedicationKnowledge', 'statu...
1    {'resourceType': 'MedicationKnowledge', 'statu...
dtype: object

In [152]:
def create_medicinal_product(data,folder=None,validation=False):
    artifact=create_pharmprod(data)
    artifact["synonym"]=[data["medicinal_product_name"]]
    artifact["code"]={"coding":[{"code":str(data["samv2_amp_id"]),"system":"http://www.belgium.be/medicinalproduct"}]}
    file_name=str(data["medicinal_product_name"]).replace("/",".").replace(".","").replace("(","").replace(")","").replace(" ","_")+ ".json"
    outcome=save_files(folder,file_name,validation,artifact)
    if outcome:
        return outcome["status"],outcome["message"]

    return artifact

In [153]:
data.iloc[0:2].apply(create_medicinal_product,axis=1,folder=OUT_FOLDER_MP)

0    {'resourceType': 'MedicationKnowledge', 'statu...
1    {'resourceType': 'MedicationKnowledge', 'statu...
dtype: object

In [154]:
data2.columns

Index(['authorization_country', 'vtm_name', 'samv2_amp_id',
       'medicinal_product_name', 'company', 'amppname', 'cnk_pub', 'cti_ext',
       'abbrevname', 'name_abbrev_fr', 'packdisplayvalue', 'espc_nl',
       'espc_fr', 'marketingauthorisationholder', 'distributor',
       'basis_substance', 'substance_modifier', 'formname', 'edqmform',
       'edqmid', 'snomedid', 'abbrform_nl', 'abbrform_fr',
       'manufactureddoseform', 'adm_doseform', 'roa_nl', 'roa_en',
       'edqm_roa_nl', 'edqm_roa_id', 'baseofstrength',
       'strenght_nominator_value_low_limit', 'strength_nominator_unit',
       'strenght_nominator_value_high_limit', 'strengthunitucum',
       'strenght_denominator_value_low_limit', 'strength_denominator_unit',
       'strengthdenomunitucum', 'atc', 'ddd', 'ddu', 'dddpropackage',
       'vmp_name', 'vmpgroupname'],
      dtype='object')

In [155]:
def create_packaged_medicinal_product(data,folder=None,validation=False):
    artifact=create_medicinal_product(data)
    artifact["synonym"]=[data["amppname"]]
    artifact["code"]={"coding":[{"code":str(data["cnk_pub"]),"system":"http://www.belgium.be/packagedmedicinalproduct"}]}
    file_name=str(data["amppname"]).replace("/",".").replace(".","").replace("(","").replace(")","").replace(" ","_")+ ".json"
    outcome=save_files(folder,file_name,validation,artifact)
    print(outcome)
    if outcome:
        return outcome["status"],outcome["message"]
    return artifact

In [156]:
#DEV: This only applies to two elements excel, remove .iloc[0:2] to apply all
#DEV: This creates a new DataFrame and then saves it to file
ndata=data2.iloc[0:2] #test purposes only
ndata[["validation_status","validation_message"]]=ndata.apply(create_packaged_medicinal_product,axis=1,folder=OUT_FOLDER_PMP,validation=True,result_type="expand")
ndata.to_csv(REPORT_FILE)

{'status': 'OK', 'message': "0 errors, 0 warnings, 1 notes\n  Information @ MedicationKnowledge.code.coding[0] (line 50, col14): Code System URI 'http://www.belgium.be/packagedmedicinalproduct' is unknown so the code cannot be validated\n"}
{'status': 'OK', 'message': "0 errors, 0 warnings, 1 notes\n  Information @ MedicationKnowledge.code.coding[0] (line 50, col14): Code System URI 'http://www.belgium.be/packagedmedicinalproduct' is unknown so the code cannot be validated\n"}


/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_83006/188322767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndata[["validation_status","validation_message"]]=ndata.apply(create_packaged_medicinal_product,axis=1,folder=OUT_FOLDER_PMP,validation=True,result_type="expand")
/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_83006/188322767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndata[["validation_status","validation_message"]]=ndata.apply(create_packaged_medicinal_product

In [161]:
def create_pharmprod_profile(data,folder=None,validation=False,ig=None):
    #print(data)
    artifact={}
    artifact["resourceType"]="MedicationKnowledge"
    artifact["meta"]= {
    "profile": [
      #"https://costateixeira.github.io/be-medication-concepts/StructureDefinition/PharmaceuticalProduct"
      "https://costateixeira.github.io/be-medication-concepts/StructureDefinition-PharmaceuticalProduct"
    ]
  }
    artifact["code"]={"coding":[]}
    artifact["code"]["coding"].append({"code":str(1200),"system":"http://www.belgium.be/pharmaceutical-ids"})
    artifact["code"]["coding"].append({"code":str(1300),"system":"http://www.edqm.eu/pharmaceutical-product-identifier-type"})

    artifact["status"]="active"
    #artifact["synonym"]=[data["vmp_name"] ]
    
    artifact["ingredient"]=[]
    denominator={"value":1} #must have 1 for ratio...
    ingredient={}

    ingredient["itemCodeableConcept"]={"coding":[{"display":data["basis_substance"]}]}
    ingredient["isActive"]=True
   # print(data["strenght_nominator_value_low_limit"],data["strength_nominator_unit"])
    numerator={"value":data["strenght_nominator_value_low_limit"],"unit":data["strength_nominator_unit"]}
    strength={"numerator":numerator}
      
    if data["strenght_denominator_value_low_limit"]=="":
     # print(data["strenght_denominator_value_low_limit"])
      denominator={"value":int(data["strenght_denominator_value_low_limit"]),"unit":data["strength_denominator_unit"]}
    strength["denominator"]=denominator
    ingredient["strength"]=strength
    artifact["ingredient"].append(ingredient)

    artifact["doseForm"] ={ ##create link with snomed or idmp
    "coding": []}

    artifact["doseForm"]["coding"].append(  {
        "code": str(data["edqmid"]),
        "system": "http://www.edqm.eu/dose-forms",
        "display": data["edqmform"]
      })

    artifact["doseForm"]["coding"].append(  {
        "code": str(data["edqmid"]),
        "system": "http://www.belgium.be/dose-forms",
        "display": data["edqmform"]
      })
    route=[]
   
    route.append( {
      "coding": [
      
      ]
    })
    route[0]["coding"].append(   {
          "code": str(data["edqm_roa_id"]),
          "system": "http://www.edqm.eu/routes",
          "display": data["roa_en"]
        })
    route[0]["coding"].append(   {
          "code": str(data["edqm_roa_id"]),
          "system": "http://www.belgium.be/routes",
          "display": data["roa_en"]
        })

    artifact["intendedRoute"]=route
    file_name=str(data["vmp_name"]).replace("/",".").replace(".","").replace("(","").replace(")","").replace(" ","_")+ ".json"
    outcome=save_files(folder,file_name,validation,artifact)
    if outcome:
        print(outcome)
        return outcome["status"],outcome["message"]

    return artifact

In [162]:
data.iloc[0:2].apply(create_pharmprod_profile,axis=1,folder=OUT_FOLDER_VMP_PROFILE,validation=True,ig="https://costateixeira.github.io/be-medication-concepts")
#data.iloc[0:2].apply(create_pharmprod_profile,axis=1,folder=OUT_FOLDER_VMP_PROFILE,validation=True,ig="/Users/joaoalmeida/Downloads/full-ig (1)/site")
#data.iloc[0:2].apply(create_pharmprod_profile,axis=1,folder=OUT_FOLDER_VMP_PROFILE,validation=True,ig="/Users/joaoalmeida/Downloads/definitions.json (1)")


{'status': 'OK', 'message': "0 errors, 1 warnings, 7 notes\n  Information @ MedicationKnowledge.meta.profile[0] (line 6, col10): Canonical URL 'https://costateixeira.github.io/be-medication-concepts/StructureDefinition-PharmaceuticalProduct' does not resolve\n  Information @ MedicationKnowledge.code.coding[0] (line 10, col14): Code System URI 'http://www.belgium.be/pharmaceutical-ids' is unknown so the code cannot be validated\n  Information @ MedicationKnowledge.code.coding[1] (line 14, col14): Code System URI 'http://www.edqm.eu/pharmaceutical-product-identifier-type' is unknown so the code cannot be validated\n  Information @ MedicationKnowledge.doseForm.coding[0] (line 44, col14): Code System URI 'http://www.edqm.eu/dose-forms' is unknown so the code cannot be validated\n  Information @ MedicationKnowledge.doseForm.coding[1] (line 49, col14): Code System URI 'http://www.belgium.be/dose-forms' is unknown so the code cannot be validated\n  Information @ MedicationKnowledge.intendedRo

0    (OK, 0 errors, 1 warnings, 7 notes\n  Informat...
1    (OK, 0 errors, 1 warnings, 7 notes\n  Informat...
dtype: object